# Remote

In [12]:
# Set working directory
domain = "fend01"
dir = "exp/e1a_linkers"
partition = "sbinlab_gpu"

remotedir = f"~/OrthoIDP/{dir}"

print(f"CONTENTS IN {remotedir} AT {domain.upper()}:")
!ssh $domain "cd $remotedir && ls"

print(f"\nCONTENTS IN {remotedir}/results AT {domain.upper()}:")
!ssh $domain "cd $remotedir/results && ls"

CONTENTS IN ~/OrthoIDP/exp/e1a_linkers AT FEND01:
data
e1a_linkers.ipynb
e1a_linkers.json
linker_sequences.fasta
results
submit.sh

CONTENTS IN ~/OrthoIDP/exp/e1a_linkers/results AT FEND01:
BA_BAdV1_linker
BtA_BtAdV3_linker
BtB_BtAdV2_linker
CA_CAdV1_linker
CA_CAdV2_linker
EA_EAdV1_linker
err
HA_HAdV18_linker
HA_HAdV31_linker
HB_HAdV11_linker
HB_SAdV27_linker
HC_HAdV5_linker
HC_SAdV34_linker
HC_SAdV43_linker
HD_HAdV10_linker
HD_HAdV19_linker
HE_SAdV22_linker
HE_SAdV25_linker
HF_HAdV40_linker
HF_HAdV41_linker
HG_HAdV52_linker
HG_SAdV7_linker
OA_BAdV2_linker
out
PC_PAdV5_linker
SA_SAdV20_linker
SA_SAdV3_linker
SB_SAdV49_linker
SC_SAdV18_linker


In [2]:
# Check availability
!ssh $domain sinfo -p $partition

/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
PARTITION   AVAIL  TIMELIMIT  NODES  STATE NODELIST
sbinlab_gpu    up   infinite      1 drain* node152
sbinlab_gpu    up   infinite      2    mix node[150-151]


In [11]:
# Check queue
!ssh $domain "squeue -u fknudsen"

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          38863362 sbinlab_g h1-0_min fknudsen  R 6-08:02:54      1 node150
          38863363 sbinlab_g h1-0_max fknudsen  R 6-08:02:54      1 node150
          38868112 sbinlab_g h1-0_con fknudsen  R 6-02:27:13      1 node150


In [27]:
# Check submit script
!ssh $domain "cat $remotedir/submit.sh"

#!/bin/bash
#SBATCH --job-name=initial
#SBATCH --partition=sbinlab_gpu
#SBATCH --array=0-5%6
#SBATCH --nodes=1
#SBATCH --cpus-per-task=1
#SBATCH --gres=gpu:1
#SBATCH -t 24:00:00
#SBATCH -o results/out
#SBATCH -e results/err


# Getting job input file
input_files=($(ls data/*.fasta))
input_file=${input_files[$SLURM_ARRAY_TASK_ID]}

# Displaying job info
echo "[`date`] STARTED Job Array ID: $SLURM_ARRAY_TASK_ID | Job ID: $SLURM_JOB_ID | Input: $input_file"

# DeiC env settings
source /groups/sbinlab/fpesce/.bashrc
conda activate openmm

# Submitting simulation
python ../../src/simulate_openmm.py -f $input_file

echo "[`date`] FINISHED Job Array ID: $SLURM_ARRAY_TASK_ID | Job ID: $SLURM_JOB_ID | Input: $input_file"


In [13]:
# Submit
!ssh $domain "cd $remotedir && sbatch submit.sh"

/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
/Users/frederikespersenknudsen/Local/OrthoIDP


In [ ]:
# Cancel job
jobid = ""
!ssh $domain scancel $jobid

In [13]:
# Copy results to BINF
!scp -r "$domain:$remotedir/results" $dir

restart.chk                                   100% 3127   348.4KB/s   00:00    
traj.dcd                                      100%   25MB   9.8MB/s   00:02    
system.xml                                    100%   15KB 776.2KB/s   00:00    
simulate.log                                  100%  561    14.7KB/s   00:00    
traj.log                                      100% 2205KB   6.7MB/s   00:00    
top.pdb                                       100% 6427   278.2KB/s   00:00    
restart.chk                                   100% 2935   825.5KB/s   00:00    
traj.dcd                                      100%   24MB   9.7MB/s   00:02    
simulate.log                                  100%  560    63.7KB/s   00:00    
system.xml                                    100%   14KB 500.6KB/s   00:00    
top.pdb                                       100% 6015   230.8KB/s   00:00    
traj.log                                      100% 2206KB   6.6MB/s   00:00    
traj.log                                

In [7]:
!ls $dir

data		  results	  submit_max.sh  submit_test.sh
idpevo_h1-0.json  submit_cons.sh  submit_min.sh


In [6]:
# Copy single specific files to BINF
filename = "r0_scan.*"
!scp "$domain:$remotedir/results/$filename" "$dir/results/$filename"

r0_scan.csv                                   100%  302KB   6.6MB/s   00:00    
r0_scan.err                                   100% 1536   337.0KB/s   00:00    
r0_scan.out                                   100% 9507     2.3MB/s   00:00    
